# Thai Wiki Language Model for Text Generation

This notebook details how you can use pretrained language model on [Thai Wikipedia Dump](https://dumps.wikimedia.org/thwiki/latest/thwiki-latest-pages-articles.xml.bz2) to generate texts.

In [1]:
# #uncomment if you are running from google colab
# !pip install sklearn_crfsuite
# !pip install https://github.com/PyThaiNLP/pythainlp/archive/dev.zip
# !pip install fastai
# !pip install emoji

In [4]:
import pandas as pd
import numpy as np
from ast import literal_eval
from tqdm import tqdm_notebook
from collections import Counter
import re

#viz
import matplotlib.pyplot as plt
import seaborn as sns

#fastai
import fastai
from fastai.text import *
from fastai.callbacks import CSVLogger

#pythainlp
from pythainlp.ulmfit import *

Download: wiki_lm_lstm
wiki_lm_lstm 0.32


100%|██████████| 1050919089/1050919089 [00:25<00:00, 41157162.35it/s]


Download: wiki_itos_lstm
wiki_itos_lstm 0.32


100%|██████████| 1530484/1530484 [00:00<00:00, 19090275.60it/s]


In [0]:
#get dummy data
imdb = untar_data(URLs.IMDB_SAMPLE)
dummy_df = pd.read_csv(imdb/'texts.csv')

#get vocab
thwiki_itos = pickle.load(open(_THWIKI_LSTM['itos_fname'],'rb'))
thwiki_vocab = fastai.text.transform.Vocab(thwiki_itos)

#dummy databunch
tt = Tokenizer(tok_func = ThaiTokenizer, lang = 'th', pre_rules = pre_rules_th, post_rules=post_rules_th)
processor = [TokenizeProcessor(tokenizer=tt, chunksize=10000, mark_fields=False),
            NumericalizeProcessor(vocab=thwiki_vocab, max_vocab=60000, min_freq=3)]
data_lm = (TextList.from_df(dummy_df, imdb, cols=['text'], processor=processor)
    .random_split_by_pct(0.2)
    .label_for_lm()
    .databunch(bs=64))


data_lm.sanity_check()

In [6]:
#check vocab size
len(data_lm.vocab.itos)

60005

In [7]:
config = dict(emb_sz=400, n_hid=1550, n_layers=4, pad_token=1, qrnn=False, tie_weights=True, out_bias=True,
             output_p=0.25, hidden_p=0.1, input_p=0.2, embed_p=0.02, weight_p=0.15)
trn_args = dict(drop_mult=0.9, clip=0.12, alpha=2, beta=1)

learn = language_model_learner(data_lm, AWD_LSTM, config=config, pretrained=False, **trn_args)

#load pretrained models
learn.load_pretrained(**_THWIKI_LSTM)

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (800 items)
x: LMTextList
xxbos   every   once   in   a   long   while   a   movie   will   come   along   that   will   be   so   xxunk   that   i   feel   xxunk   to   xxunk   people .   if   i   labor   all   my   days   and   i   can   save   but   one   soul   from   xxunk   this   movie ,   how   great   will   be   my   joy .   
   
   where   to   begin   my   xxunk   of   pain .   for   xxunk ,   there   was   a   musical   xxunk   every   five   minutes .   there   was   no   character   development .   every   character   was   a   stereotype .   we   had   xxunk   guy ,   fat   guy   who   xxunk   xxunk ,   xxunk   foreign   guy ,   etc .   the   script   xxunk   as   if   it   were   being   written   as   the   movie   was   being   shot .   the   production   value   was   so   xxunk   low   that   it   xxunk   like   i   was   xxunk   a   junior   high   video   presentation .   have   the   directors ,   producers

In [24]:
print(learn.predict('กาลครั้งหนึ่งนานมาแล้ว ', 200, temperature=0.8, min_p=0.005, sep = ''))

กาลครั้งหนึ่งนานมาแล้ว คุณวันจันทร์ได้รับการอุปการะจากแม่เธอ 
 
 วันต่อมา เธอได้พบกับ "อาเธอร์ โอลด์เดน" เด็กหนุ่มที่มีบุคลิกคล้ายกับ "เอมิลี" ซึ่งเป็นน้องสาวของนาง เขาจึงได้รับการเลี้ยงดูจาก "อาเธอร์ ดีอา" และเป็นผู้ที่คอยดูแลเธออยู่เสมอ เธอได้แนะนำให้เธอเป็นผู้หญิง 
 
 โดยมี "เอลซ่า" ซึ่งเป็นทายาทของ "อาเธอร์ ยูจีน" ผู้เป็นสามีของเขา และเคยช่วยยูนิตที่ถูกส่งตัวไปประจำอยู่ที่ดินแดนแห่งนี้ เธอได้พบกับ "ยูลิสซิส เกรย์ เอลิซาเบธ เอลลิส " (เอมิลี่ ไอเซนฮา) ซึ่งเป็นชาว "เผ่าเอลฟ์ " และเธอก็ไม่ค่อยมีบุตร แต่เธอก็ได้รับความช่วยเหลือจาก "
